# 패키지 로딩

In [1]:
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

# 데이터 로딩 및 전처리

## 독립변수

### HSI (월별)

In [25]:
# 데이터 로딩
HSI = pd.read_csv("./구리 수요/(수요) Housing_Starts_Index_May_2023.csv")

# 필요 없는 열 제거
HSI = HSI.drop("Unnamed: 2", axis = 1)

# 날짜데이터 pandas datetime으로 변경
HSI["Date"] = pd.to_datetime(HSI["Date"])

# 열 이름 변경
HSI.rename(columns = {'Value' : 'HSI_value'}, inplace = True)

HSI

,Date,HSI_value
0,1990-01-31,1551
1,1990-02-28,1437
2,1990-03-31,1289
3,1990-04-30,1248
4,1990-05-31,1212
...,...,...
396,2023-01-31,1340
397,2023-02-28,1436
398,2023-03-31,1380
399,2023-04-30,1340


### CCI (월별)

In [3]:
# 데이터 로딩
CCI = pd.read_csv("./구리 수요/(수요) Consumer confidence index (CCI).csv")

# 날짜데이터 pandas datetime으로 변경
CCI["TIME"] = pd.to_datetime(CCI["TIME"])

# 열 이름 변경
CCI.rename(columns = {'TIME' : 'Date'}, inplace = True)
CCI.rename(columns = {'Value' : 'CCI_value'}, inplace = True)

CCI

,Date,CCI_value
0,1990-01-01,102.05330
1,1990-02-01,101.93030
2,1990-03-01,101.84770
3,1990-04-01,101.73750
4,1990-05-01,101.60550
...,...,...
396,2023-01-01,97.72659
397,2023-02-01,98.16353
398,2023-03-01,98.40943
399,2023-04-01,98.43763


### IPI (월별)

In [4]:
# 데이터 로딩
IPI = pd.read_csv("./구리 수요/(수요) Industrial Production Total Index_INDPRO.csv")

# 날짜데이터 pandas datetime으로 변경
IPI["DATE"] = pd.to_datetime(IPI["DATE"])

# 열 이름 변경
IPI.rename(columns = {'DATE' : 'Date'}, inplace = True)
IPI.rename(columns = {'INDPRO' : 'IPI_value'}, inplace = True)

IPI

,Date,IPI_value
0,1990-01-01,61.6440
1,1990-02-01,62.2108
2,1990-03-01,62.5068
3,1990-04-01,62.3597
4,1990-05-01,62.5474
...,...,...
396,2023-01-01,102.5080
397,2023-02-01,102.5023
398,2023-03-01,102.6521
399,2023-04-01,103.1748


### GDP (분기별 -> 월별)

In [62]:
# 데이터 로딩
GDPC = pd.read_csv("./구리 수요/(수요) 분기별 Real Gross Domestic Product (GDPC1)1990-2023_JAN.csv")

# 날짜데이터 pandas datetime으로 변경
GDPC["DATE"] = pd.to_datetime(GDPC["DATE"])

# datetime에서 연, 월 추출
GDPC["Year"] = GDPC["DATE"].dt.year
GDPC["Month"] = GDPC["DATE"].dt.month

# 열 이름 변경
GDPC.rename(columns = {'GDPC1' : 'GDPC_value'}, inplace = True)

# 분기데이터 월별데이터로 늘리기
## 이중 for문으로 한 연도의 분기별 데이터를 각각 3개씩 늘리고, 각 분기에 해당하는 월별데이터로 교체
year_list = []

# GDPC의 연도별 데이터의 고유값만큼 반복문 실행
for i in GDPC["Year"].unique():
    
    # i에 할당된 연도와 일치하는 데이터를 temp 변수로 가져옴
    temp = GDPC[GDPC["Year"] == i]
    
    # temp에 할당된 월별 데이터만큼 반복문 실행
    for j in temp["Month"].unique():
        
        # j에 할당된 연도와 일치하는 데이터를 temp2 변수로 가져옴
        temp2 = temp[temp["Month"] == j]
        temp2.reset_index(drop=True, inplace=True)
        
        # loc 함수를 통해 해당 월 기준 다음달, 다다음달 데이터를 행 단위로 추가
        ## 기존 DataFrame의 열과 일치하도록 DATE, GDPC_value, Year, Month 복제
        ## formatted string로 DATE 표기 타입을 통일
        
        ### 다음달 데이터
        temp2.loc[1] = [f'{i}-{j+1:02}-01', temp2.at[temp2.index[0], "GDPC_value"], temp2.at[temp2.index[0], "Year"], temp2.at[temp2.index[0], "Month"]+1]
        
        ### 다다음달 데이터
        temp2.loc[2] = [f'{i}-{j+2:02}-01', temp2.at[temp2.index[0], "GDPC_value"], temp2.at[temp2.index[0], "Year"], temp2.at[temp2.index[0], "Month"]+2]
        
        # 생성한 3달치 월별 데이터를 리스트에 추가
        year_list.append(temp2)

# 생성한 월별 데이털 리스트를 행 단위로 DataFrame 병합
a = pd.concat(year_list)

# 날짜데이터 pandas datetime으로 변경
a["DATE"] = pd.to_datetime(a.DATE)

# 날짜 기준 오름차순 정렬
a = a.sort_values("DATE")

# 행 번호 초기화
a.reset_index(drop=True, inplace=True)

# 열 이름 변경
a.rename(columns = {'DATE' : 'Date'}, inplace = True)

GDPC = a[["Date", "GDPC_value"]]
GDPC

,Date,GDPC_value
0,1990-01-01,9364.259
1,1990-02-01,9364.259
2,1990-03-01,9364.259
3,1990-04-01,9398.243
4,1990-05-01,9398.243
...,...,...
394,2022-11-01,20182.491
395,2022-12-01,20182.491
396,2023-01-01,20246.439
397,2023-02-01,20246.439


### 구리 가격 (일별 -> 월별)

In [61]:
# 데이터 로딩
copper = pd.read_csv("./구리 수요/(수요) copper-prices-historical-chart-data.csv")

# 날짜데이터 pandas datetime으로 변경
copper["date"] = pd.to_datetime(copper["date"])

# 1995년 1월부터 2023년 3월까지 데이터만 가져오기
copper = copper[copper["date"] >= "1995-01-01"][copper["date"] <= "2023-03-31"].reset_index(drop=True)

# 년 - 월 컬럼 추가 후 월별 평균값 계산
copper['month_year'] = copper['date'].dt.to_period('M')
copper = copper.groupby("month_year").mean("value").reset_index(drop=True)
copper.columns = ["Copper price"]

copper

,Copper price
0,1.380976
1,1.322179
2,1.351283
3,1.328911
4,1.260091
...,...
334,3.656582
335,3.817891
336,4.118691
337,4.082400


## 종속변수: 전세계 구리 수요

In [64]:
# 데이터 로딩
DEP = pd.read_csv("./구리 수요/(종속 변수) 230622 Global Copper Demand.csv")

# 열 이름 변경
DEP.columns = ["Date", "PX_LAST"]

DEP

,Date,PX_LAST
0,1995-01-31,969917.000
1,1995-02-28,960039.000
2,1995-03-31,1043928.000
3,1995-04-30,1022904.000
4,1995-05-31,979813.000
...,...,...
334,2022-11-30,2237807.240
335,2022-12-31,2148356.118
336,2023-01-31,2121282.622
337,2023-02-28,1975076.391


# 데이터 병합

In [65]:
# HSI, CCI, IPI, GPDC 병합 (1995년 자료부터 가져오기)
total = pd.concat([HSI[["HSI_value"]], CCI[["CCI_value"]], IPI[["IPI_value"]], GDPC[["GDPC_value"]]], axis = 1)[60:].reset_index(drop=True)

# 구리가격 데이터, 종속변수 결합
total = pd.concat([total, copper, DEP], axis=1)

# 결측치 있는 행 제거
total = total[:-2]

# 열 순서 변경
total = total[['Date', 'HSI_value', 'CCI_value', 'IPI_value', 'GDPC_value', 'Copper price', 'PX_LAST']]

total

,Date,HSI_value,CCI_value,IPI_value,GDPC_value,Copper price,PX_LAST
0,1995-01-31,1407,100.83360,71.2762,10550.251,1.380976,969917.000
1,1995-02-28,1316,100.94720,71.1904,10550.251,1.322179,960039.000
2,1995-03-31,1249,101.02100,71.2891,10550.251,1.351283,1043928.000
3,1995-04-30,1267,100.98750,71.1623,10581.723,1.328911,1022904.000
4,1995-05-31,1314,100.98590,71.5044,10581.723,1.260091,979813.000
...,...,...,...,...,...,...,...
334,2022-11-30,1427,96.77289,103.0707,20182.491,3.656582,2237807.240
335,2022-12-31,1357,97.26849,101.4848,20182.491,3.817891,2148356.118
336,2023-01-31,1340,97.72659,102.5080,20246.439,4.118691,2121282.622
337,2023-02-28,1436,98.16353,102.5023,20246.439,4.082400,1975076.391


# 데이터 저장

In [66]:
total.to_csv("./구리 수요/수요 변수_최종.csv", index=False)